In [22]:
print('Hello LangChain')

Hello LangChain


In [23]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [24]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [25]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001F7822BB200> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F78283B2F0> root_client=<openai.OpenAI object at 0x000001F78283BFE0> root_async_client=<openai.AsyncOpenAI object at 0x000001F78283B950> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [26]:
try:    # llm 호출
    response = llm.invoke(prompt_text) 
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

응답: LangServe는 개발자가 대규모 언어 모델(LLM)을 쉽게 배포하고 관리할 수 있도록 지원하는 오픈 소스 라이브러리입니다. LangServe를 사용하면 개발자는 언어 모델을 API 서버로 쉽게 전환하여 다양한 애플리케이션에서 언어 모델을 활용할 수 있습니다.

LangServe의 주요 기능은 다음과 같습니다:

1. **LLM 통합**: LangServe는 다양한 언어 모델 라이브러리(LLaMA, Hugging Face 등)와 통합됩니다. 이를 통해 개발자는 자신이 선호하는 언어 모델을 선택하여 사용할 수 있습니다.

2. **API 서버**: LangServe를 사용하면 언어 모델을 API 서버로 쉽게 배포할 수 있습니다. 이를 통해 웹 애플리케이션, 모바일 앱 등 다양한 클라이언트에서 언어 모델을 호출하여 사용할 수 있습니다.

3. **관리 및 모니터링**: LangServe는 언어 모델의 성능을 모니터링하고 관리할 수 있는 기능을 제공합니다. 개발자는 모델의 응답 시간, 오류율 등을 모니터링하여 모델의 성능을 최적화할 수 있습니다.

4. **보안**: LangServe는 데이터 암호화, 접근 제어 등 보안 기능을 제공하여 언어 모델을 안전하게 배포할 수 있도록 지원합니다.

5. **확장성**: LangServe는 수평 확장성을 지원하여 대규모 트래픽을 처리할 수 있습니다. 이를 통해 개발자는 대규모 애플리케이션에서도 언어 모델을 안정적으로 사용할 수 있습니다.

LangServe의 사용 사례는 매우 다양합니다. 예를 들어, 챗봇, 가상 비서, 언어 번역, 텍스트 요약 등 다양한 자연어 처리 애플리케이션에서 LangServe를 활용할 수 있습니다.

LangServe를 사용하는 방법은 상대적으로 간단합니다. 개발자는 LangServe 라이브러리를 설치하고, 언어 모델을 선택하여 API 서버를 설정하면 됩니다. 이후에는 클라이언트에서 API를 호출하여 언어 모델을 사용할 수 있습니다.

LangServe는 개발자가 언어 모델을 쉽게 배포하고 

### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [27]:
from langchain_core.prompts import PromptTemplate


prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 한글로 설명해주세요.
    """)
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question. \n    <Question>: {input}에 대해 쉽게 한글로 설명해주세요.\n    ')

In [28]:
# chain 연결 (LCEL)
chain = prompt | llm

### LCEL
* Prompt + LLM + OuterParser를 Chain으로 연결하기

In [29]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL)
output_parser = StrOutputParser()
chain2 = prompt | llm | output_parser

In [30]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 방식과 유사합니다. 컴퓨터가 데이터를 통해 스스로 규칙을 찾고, 그 규칙을 바탕으로 미래의 데이터를 예측하거나 분류하는 것입니다.

쉽게 말해, 인공지능 모델이 학습하는 과정을 다음과 같이 비유할 수 있습니다.

1. **데이터 수집**: 학생이 공부를 시작할 때, 책이나 자료를 많이 모아야 하듯이, 인공지능 모델도 학습을 위해 많은 데이터를 수집합니다.

2. **데이터 준비**: 수집한 데이터를 정리하고, 필요한 경우 데이터를 변환하거나 가공합니다. 이는 학생이 공부 자료를 정리하고 이해하기 쉽게 만드는 과정과 비슷합니다.

3. **모델 훈련**: 이 데이터를 바탕으로 인공지능 모델이 스스로 규칙이나 패턴을 찾습니다. 예를 들어, 고양이와 강아지의 사진을 많이 보여주면서 '이것은 고양이야', '이것은 강아지야'라고 알려주는 겁니다. 모델은 이 과정을 통해 고양이와 강아지를 구별하는 규칙을 스스로 터득합니다.

4. **평가**: 훈련이 끝난 후, 모델이 얼마나 잘 학습했는지 평가합니다. 새로운 사진들을 모델에 보여주고, 모델이 올바르게 분류하는지 확인하는 과정입니다.

5. **반복**: 모델의 성능이 만족스럽지 않다면, 더 많은 데이터를 수집하거나, 모델의 구조를 바꾸는 등 다양한 방법으로 학습 과정을 반복합니다.

이를 통해 인공지능 모델은 주어진 데이터를 기반으로 하여, 새로운 데이터에 대해서도 정확한 예측이나 분류를 할 수 있게 됩니다.

예를 들어, 음성 인식 인공지능 모델을 개발한다고 가정해 봅시다. 이 모델에게 수많은 사람들의 목소리를 녹음한 데이터를 제공하고, 그것이 어떤 단어인지, 어떤 문장인지를 학습시킵니다. 그러면 이 모델은 새로운 목소리를 듣고 그 내용을 예측할 수 있게 됩니다.

이처럼 인공지능 모델의 학습은 데이터로부터 패턴을 발견하고, 그 패턴을 바탕으로 미래의 데이터를 이해하는 과정이라고 할 수

In [31]:
# chain 호출
try:
    result = chain2.invoke({"input": "LangChain의 Product(제품)에는 어떤 것들이 있나요?"})
    print(type(result))
    print(result) # output parser라서 스트링으로 바로 나옴
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 인공지능(AI) 기술을 기반으로 하는 플랫폼으로, 다양한 제품과 서비스를 제공하고 있습니다. LangChain의 주요 제품은 다음과 같습니다.

1. **LangChain 플랫폼**: LangChain은 개발자들이 AI 기반 애플리케이션을 쉽게 구축할 수 있도록 지원하는 플랫폼입니다. 이 플랫폼은 자연어 처리(NLP), 기계 학습(ML), 딥러닝 등 다양한 AI 기술을 통합하여 제공합니다.

2. **LangChain API**: LangChain API는 개발자들이 자사의 애플리케이션에 AI 기능을 통합할 수 있도록 지원하는 API입니다. 이 API를 통해 개발자들은 자연어 처리, 이미지 인식, 음성 인식 등 다양한 AI 기능을 사용할 수 있습니다.

3. **LangChain Studio**: LangChain Studio는 AI 모델을 개발하고 훈련시키는 데 사용되는 개발 환경입니다. 이 환경은 개발자들이 AI 모델을 쉽게 생성, 훈련, 배포할 수 있도록 지원합니다.

4. **LangChain 모델**: LangChain은 다양한 사전 훈련된 AI 모델을 제공합니다. 이 모델들은 자연어 처리, 컴퓨터 비전, 음성 인식 등 다양한 분야에서 활용될 수 있습니다.

5. **LangChain 에이전트**: LangChain 에이전트는 사용자가 자연어로 요청을 하면 그에 따라 작업을 수행하는 AI 에이전트입니다. 사용자는 에이전트를 통해 다양한 작업을 요청할 수 있으며, 에이전트는 그에 따라 적절한 답변을 제공합니다.

이러한 제품과 서비스를 통해 LangChain은 개발자와 기업이 AI 기술을 쉽게 활용할 수 있도록 지원하고 있습니다.


### Runnable의 stream() 함수 호출

In [32]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})

    # 스트리밍 출력
    # print(answer)

    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")



인공지능 모델의 학습 원리를 쉽게 설명해 드리겠습니다.

**인공지능 모델의 학습 원리**

인공지능 모델의 학습은 데이터를 통해 모델이 스스로 학습하고, 예측 또는 분류를 잘하도록 만드는 과정입니다. 이를 위해 다음과 같은 단계로 학습이 이루어집니다.

### 1. 데이터 수집

모델을 학습시키기 위해서는 많은 양의 데이터가 필요합니다. 이 데이터는 문제의 유형에 따라 달라지며, 예를 들어 이미지 분류 모델을 만든다면 다양한 이미지 데이터를 수집해야 합니다.

### 2. 데이터 전처리

수집된 데이터는 학습에 적합한 형태로 변환되어야 합니다. 예를 들어, 이미지 데이터의 경우, 이미지의 크기를 조정하거나, 픽셀 값을 정규화하는 등의 작업이 필요합니다.

### 3. 모델 선택

적합한 모델을 선택해야 합니다. 모델의 종류로는 신경망, 결정 트리, 서포트 벡터 머신 등 다양한 모델이 있으며, 문제의 유형과 데이터의 특성에 따라 적절한 모델을 선택해야 합니다.

### 4. 모델 학습

선택된 모델에 데이터를 입력하여 모델을 학습시킵니다. 이 과정에서는 모델이 데이터의 패턴을 인식하고, 예측 또는 분류를 잘하도록 모델의 파라미터를 조정합니다.

### 5. 손실 함수 정의

모델의 성능을 평가하기 위해 손실 함수를 정의합니다. 손실 함수는 모델의 예측과 실제 값 사이의 차이를 측정합니다. 예를 들어, 분류 문제에서는 크로스 엔트로피 손실 함수를 사용합니다.

### 6. 최적화 알고리즘 적용

모델의 파라미터를 조정하여 손실 함수를 최소화하는 최적화 알고리즘을 적용합니다. 대표적인 최적화 알고리즘에는 경사 하강법, Adam, RMSProp 등이 있습니다.

### 7. 모델 평가

학습된 모델의 성능을 평가합니다. 이를 위해 검증 데이터를 사용하여 모델의 성능을 측정하고, 필요에 따라 모델을 수정하거나 학습을 반복합니다.

### 8. 모델 배포

학습된 모델을 배포하여 실제 문제에 적용합니다.

**핵심 개념**

* **오버피팅**: 모델이 학습 데이터에 

### Multi Chain
* 첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.
* 두개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기